In [1]:
!pip -q install transformers
!pip -q install sentencepiece
!pip -q install accelerate

In [2]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer

# Hugging Face model_path
model_path = 'psmathur/orca_mini_3b'
tokenizer = LlamaTokenizer.from_pretrained(model_path)
model = LlamaForCausalLM.from_pretrained(
    model_path, torch_dtype=torch.float16, device_map='auto',
)


#generate text function
def generate_text(system, instruction, input=None):

    if input:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Input:\n{input}\n\n### Response:\n"
    else:
        prompt = f"### System:\n{system}\n\n### User:\n{instruction}\n\n### Response:\n"

    tokens = tokenizer.encode(prompt)
    tokens = torch.LongTensor(tokens).unsqueeze(0)
    tokens = tokens.to('cuda')

    instance = {'input_ids': tokens,'top_p': 1.0, 'temperature':0.7, 'generate_len': 1024, 'top_k': 50}

    length = len(tokens[0])
    with torch.no_grad():
        rest = model.generate(
            input_ids=tokens,
            max_length=length+instance['generate_len'],
            use_cache=True,
            do_sample=True,
            top_p=instance['top_p'],
            temperature=instance['temperature'],
            top_k=instance['top_k']
        )
    output = rest[0][length:]
    string = tokenizer.decode(output, skip_special_tokens=True)
    return f'[!] Response: {string}'

# Sample Test Instruction Used by Youtuber Sam Witteveen https://www.youtube.com/@samwitteveenai
system = 'You are an AI assistant that follows instruction extremely well. Help as much as you can.'
instruction = 'Write a letter to Sam Altman, CEO of OpenAI, requesting him to convert GPT4 a private model by OpenAI to an open source project'
print(generate_text(system, instruction))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

[!] Response: Dear Sam Altman,

I am writing to request that OpenAI makes GPT4, a private model developed by the company, a public open-source project. As an AI assistant, I understand the importance of open access to data and models in the field of AI.

By making GPT4 a public open-source project, individuals and organizations from all over the world would be able to use and improve the model, leading to greater innovation and progress in the field of AI. This would also ensure that the model is not limited to the proprietary purposes of private companies, but can be used for the greater good of society.

OpenAI has a history of releasing AI models in their beta form, which has led to significant advancements in the field of AI. By releasing GPT4 as a public open-source project, OpenAI can continue to make contributions to the field and benefit from the collective knowledge and expertise of the community.

I urge OpenAI to consider this request and take the necessary steps to make GPT